# 数据操作

:label:`sec_ndarray`

为了能够完成各种数据操作，我们需要某种方法来存储和操作数据。
通常，我们需要做两件重要的事：（1）获取数据；（2）将数据读入计算机后对其进行处理。
如果没有某种方法来存储数据，那么获取数据是没有意义的。

首先，我们介绍$n$维数组，也称为*张量*（tensor）。
使用过Python中NumPy计算包的读者会对本部分很熟悉。
无论使用哪个深度学习框架，它的*张量类*（在MXNet中为`ndarray`，
在PyTorch和TensorFlow中为`Tensor`）都与Numpy的`ndarray`类似。
但深度学习框架又比Numpy的`ndarray`多一些重要功能：
首先，GPU很好地支持加速计算，而NumPy仅支持CPU计算；
其次，张量类支持自动微分。
这些功能使得张量类更适合深度学习。
如果没有特殊说明，本书中所说的张量均指的是张量类的实例。

首先，我们导入 torch。请注意，虽然它被称为 PyTorch，但我们应该导入 torch 而不是 pytorch。

In [2]:
import torch

[**张量表示由一个数值组成的数组，这个数组可能有多个维度**]。
具有一个轴的张量对应数学上的*向量*（vector）；
具有两个轴的张量对应数学上的*矩阵*（matrix）；
具有两个轴以上的张量没有特殊的数学名称。

In [3]:
x = torch.arange(12)
print(x)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])


In [4]:
print(x.shape)

torch.Size([12])


In [6]:
x = x.reshape(3, 4)
print(x)
x.numel()

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


12

In [7]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

有时我们想通过从某个特定的概率分布中随机采样来得到张量中每个元素的值。
例如，当我们构造数组来作为神经网络中的参数时，我们通常会随机初始化参数的值。
以下代码创建一个形状为（3,4）的张量。
其中的每个元素都从均值为0、标准差为1的标准高斯分布（正态分布）中随机采样。


In [8]:
torch.randn(3, 4)

tensor([[ 0.0355, -0.0448,  0.6450, -0.2751],
        [ 0.9710,  0.1723,  1.3996, -0.9348],
        [ 1.3408,  0.8212, -0.5997,  0.0969]])

我们还可以[**通过提供包含数值的Python列表（或嵌套列表），来为所需张量中的每个元素赋予确定值**]。
在这里，最外层的列表对应于轴0，内层的列表对应于轴1。


In [9]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [3, 4, 1, 2]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [3, 4, 1, 2]])

## 运算符

我们的兴趣不仅限于读取数据和写入数据。
我们想在这些数据上执行数学运算，其中最简单且最有用的操作是*按元素*（elementwise）运算。
它们将标准标量运算符应用于数组的每个元素。
对于将两个数组作为输入的函数，按元素运算将二元运算符应用于两个数组中的每对位置对应的元素。
我们可以基于任何从标量到标量的函数来创建按元素函数。

在数学表示法中，我们将通过符号$f: \mathbb{R} \rightarrow \mathbb{R}$
来表示*一元*标量运算符（只接收一个输入）。
这意味着该函数从任何实数（$\mathbb{R}$）映射到另一个实数。
同样，我们通过符号$f: \mathbb{R}, \mathbb{R} \rightarrow \mathbb{R}$
表示*二元*标量运算符，这意味着该函数接收两个输入，并产生一个输出。
给定同一形状的任意两个向量$\mathbf{u}$和$\mathbf{v}$和二元运算符$f$，
我们可以得到向量$\mathbf{c} = F(\mathbf{u},\mathbf{v})$。
具体计算方法是$c_i \gets f(u_i, v_i)$，
其中$c_i$、$u_i$和$v_i$分别是向量$\mathbf{c}$、$\mathbf{u}$和$\mathbf{v}$中的元素。
在这里，我们通过将标量函数升级为按元素向量运算来生成向量值
$F: \mathbb{R}^d, \mathbb{R}^d \rightarrow \mathbb{R}^d$。

对于任意具有相同形状的张量，
[**常见的标准算术运算符（`+`、`-`、`*`、`/`和`**`）都可以被升级为按元素运算**]。
我们可以在同一形状的任意两个张量上调用按元素操作。
在下面的例子中，我们使用逗号来表示一个具有5个元素的元组，其中每个元素都是按元素操作的结果。


In [10]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y  # **运算符是求幂运算

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

[**我们也可以把多个张量*连结*（concatenate）在一起**]，
把它们端对端地叠起来形成一个更大的张量。
我们只需要提供张量列表，并给出沿哪个轴连结。
下面的例子分别演示了当我们沿行（轴-0，形状的第一个元素）
和按列（轴-1，形状的第二个元素）连结两个矩阵时，会发生什么情况。
我们可以看到，第一个输出张量的轴-0长度（$6$）是两个输入张量轴-0长度的总和（$3 + 3$）；
第二个输出张量的轴-1长度（$8$）是两个输入张量轴-1长度的总和（$4 + 4$）。

In [13]:
X = torch.arange(12, dtype = float).reshape(3, 4)
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2,1]])
torch.cat((X, Y), dim = 0), torch.cat((X, Y), dim = 1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]], dtype=torch.float64),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]], dtype=torch.float64))

[**对张量中的所有元素进行求和，会产生一个单元素张量。**]

In [14]:
X.sum()

tensor(66., dtype=torch.float64)

## 广播机制
:label:`subsec_broadcasting`

在上面的部分中，我们看到了如何在相同形状的两个张量上执行按元素操作。
在某些情况下，[**即使形状不同，我们仍然可以通过调用
*广播机制*（broadcasting mechanism）来执行按元素操作**]。
这种机制的工作方式如下：首先，通过适当复制元素来扩展一个或两个数组，
以便在转换之后，两个张量具有相同的形状。
其次，对生成的数组执行按元素操作。

在大多数情况下，我们将沿着数组中长度为1的轴进行广播，如下例子：

In [15]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [16]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

## 索引和切片

就像在任何其他Python数组中一样，张量中的元素可以通过索引访问。
与任何Python数组一样：第一个元素的索引是0，最后一个元素索引是-1；
可以指定范围以包含第一个元素和最后一个之前的元素。

如下所示，我们[**可以用`[-1]`选择最后一个元素，可以用`[1:3]`选择第二个和第三个元素**]：


In [17]:
X[-1], X[1: 3]

(tensor([ 8.,  9., 10., 11.], dtype=torch.float64),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]], dtype=torch.float64))

In [18]:
X[1, 2] = 9
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  9.,  7.],
        [ 8.,  9., 10., 11.]], dtype=torch.float64)

如果我们想[**为多个元素赋值相同的值，我们只需要索引所有元素，然后为它们赋值。**]
例如，`[0:2, :]`访问第1行和第2行，其中“:”代表沿轴1（列）的所有元素。
虽然我们讨论的是矩阵的索引，但这也适用于向量和超过2个维度的张量。


In [21]:
X[0:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]], dtype=torch.float64)

## 节省内存

[**运行一些操作可能会导致为新结果分配内存**]。
例如，如果我们用`Y = X + Y`，我们将取消引用`Y`指向的张量，而是指向新分配的内存处的张量。

在下面的例子中，我们用Python的`id()`函数演示了这一点，
它给我们提供了内存中引用对象的确切地址。
运行`Y = Y + X`后，我们会发现`id(Y)`指向另一个位置。
这是因为Python首先计算`Y + X`，为结果分配新的内存，然后使`Y`指向内存中的这个新位置。


In [27]:
before = id(Y)
Y = Y + X
id(Y) == before

False

[**如果在后续计算中没有重复使用`X`，
我们也可以使用`X[:] = X + Y`或`X += Y`来减少操作的内存开销。**]


In [28]:
before = id(X)
X[:] = X + Y
id(X) == before

True

## 转换为其他Python对象
将深度学习框架定义的张量[**转换为NumPy张量（`ndarray`）**]很容易，反之也同样容易。
torch张量和numpy数组将共享它们的底层内存，就地操作更改一个张量也会同时更改另一个张量。


In [29]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

要(**将大小为1的张量转换为Python标量**)，我们可以调用`item`函数或Python的内置函数。


In [30]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

## 小结

* 深度学习存储和操作数据的主要接口是张量（$n$维数组）。它提供了各种功能，包括基本数学运算、广播、索引、切片、内存节省和转换其他Python对象。

## 练习

1. 运行本节中的代码。将本节中的条件语句`X == Y`更改为`X < Y`或`X > Y`，然后看看你可以得到什么样的张量。
1. 用其他形状（例如三维张量）替换广播机制中按元素操作的两个张量。结果是否与预期相同？
